In [1]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
print(CUDA, TORCH)

cu121 2.5.1


In [2]:
# !gdown 1W90lq3I_5RY-Ff48hjR3QoqBzHFAxOiR
# from google.colab import drive
# drive.mount('/content/drive')
# !mkdir /content/drive/MyDrive/IDSC

In [3]:
# !unzip /content/IDSC2025_AnXplore03.zip -d /content/drive/MyDrive/IDSC/

In [4]:
from google.colab import drive
drive.mount('/content/drive')

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install loguru==0.7.2
!pip install autoflake==2.3.0
!pip install pytest==8.0.1
!pip install meshio==5.3.5
!pip install h5py==3.10.0

import os
import os.path as osp
import shutil
from itertools import product
import meshio
from typing import List
from torch_geometric.data import Dataset as BaseDataset
from torch_geometric.data import Data
import torch
import numpy as np
from torch_geometric.loader import DataLoader

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.loader import DataLoader
#from DataLoader import Dataset
#from EncoderDecoder import EncodeProcessDecode
#from Utils import L2Loss, Simulator
#from Epoch import TrainEpoch

writer = SummaryWriter("tensorboard")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(device)
folder_path = '/content/drive/MyDrive/IDSC/4Students_AnXplore03/'
#folder_path ="/Users/ludoviclepic/Downloads/4Students_AnXplore03/"
# folder_path = '../input/anevrisme/4Students_AnXplore03/'
dataset = Dataset(folder_path)

train_loader = DataLoader(
    dataset=dataset,
    batch_size=5,  # Use a smaller batch size to reduce memory usage
    shuffle=True,
    num_workers=0,  # Reduce the number of workers to avoid semaphore issues
    pin_memory=False,  # Pin memory to speed up data transfer to GPU
    persistent_workers=False  # Disable persistent workers to avoid semaphore issues
)

model = EncodeProcessDecode(
    node_input_size=6,
    edge_input_size=3,
    message_passing_num=5,
    hidden_size=32,
    node_output_size=6,
    edge_output_size=3,
) #
loss = L2Loss() #
simulator = Simulator(
    node_input_size=6,
    edge_input_size=3,
    output_size=6,
    feature_index_start=0,
    feature_index_end=4,
    output_index_start=0,
    output_index_end=6,
    node_type_index=5,
    batch_size=5,
    model=model,
    device=device,
    model_dir="Groupe2/checkpoint/simulator.pth",
    time_index=4
) #
optimizer = torch.optim.Adam(simulator.parameters(), lr=0.0001)

train_epoch = TrainEpoch(
    model=simulator,
    loss=loss,
    optimizer=optimizer,
    parameters={},
    device=device,
    verbose=True,
    starting_step=0,
    use_sub_graph=False,
    accumulation_steps=4  # Set accumulation steps for gradient accumulation
)

import os

# Ensure the Drive folder exists
drive_checkpoint_folder = '/content/drive/MyDrive/Groupe2/simulator_checkpoints'
os.makedirs(drive_checkpoint_folder, exist_ok=True)

for i in range(0, 15):
    print("\nEpoch: {}".format(i))
    print("=== Training ===")
    train_loss = train_epoch.run(train_loader, writer, "model.pth")
    print(f"Epoch {i} completed with train_loss: {train_loss}")

    # Save the model directly to Google Drive
    model_path = f"{drive_checkpoint_folder}/simulator_epoch_{i}.pth"
    torch.save(simulator.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    writer.add_scalar("Loss/train/mean_value_per_epoch", train_loss, i)
    writer.flush()
    writer.file_writer.flush()  # Clear the writer's logs

writer.close()
torch.cuda.empty_cache()
gc.collect()  # Force garbage collection


Epoch cuda

Epoch: 0
=== Training ===
train:   7%|▋         | 107/1628 [03:18<48:13,  1.90s/it]